In [9]:
!pip install qiskit qiskit-aer matplotlib pylatexenc --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [10]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt


In [13]:
# 1. Quantum Circuit Creation
# ------------------------------------------------------
# Create a 2-qubit circuit with 2 classical bits
qc = QuantumCircuit(2, 2)

# Apply a Hadamard gate to qubit 0
qc.h(0)

# Apply a CNOT gate (control: qubit 0, target: qubit 1)
qc.cx(0, 1)

# Measure both qubits
qc.measure([0, 1], [0, 1])

# Safely draw the circuit (fallback if pylatexenc is missing)
try:
    qc.draw('mpl')
    plt.show()
except Exception as e:
    print("⚠️ Matplotlib drawer not available, showing text diagram instead:\n")
    print(qc.draw())

⚠️ Matplotlib drawer not available, showing text diagram instead:

     ┌───┐     ┌─┐   
q_0: ┤ H ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
          └───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1 


In [14]:
# 3. Simulation using AerSimulator
# ------------------------------------------------------
sim = AerSimulator()
t_qc = transpile(qc, sim)
job = sim.run(t_qc, shots=1024)
result = job.result()
counts = result.get_counts()

# Display results
print("\n===== Simulation Results =====")
print(counts)
plot_histogram(counts)
plt.show()


===== Simulation Results =====
{'00': 517, '11': 507}


In [17]:
# Try to import QASM3 dumps; if not available, we'll fall back later
try:
    from qiskit.qasm3 import dumps as qasm3_dumps
except Exception as e:
    qasm3_dumps = None
    print("Note: qiskit.qasm3.dumps unavailable in this environment (will use fallback).")

# ---------------------------
# Utility: robust QASM exporter
# ---------------------------
def get_qasm(circuit):
    """
    Return a QASM string for `circuit`.
    Tries qiskit.qasm3.dumps (QASM3) first, then QuantumCircuit.qasm() if available.
    Raises RuntimeError if no exporter is available.
    """
    # Prefer QASM3 dumps if available
    if qasm3_dumps is not None:
        try:
            return qasm3_dumps(circuit)
        except Exception as e:
            # if dumps fails, continue to fallback
            print("qasm3.dumps failed:", e)

    # Fallback: try legacy .qasm() method (may not exist in Qiskit 2.x depending on install)
    try:
        return circuit.qasm()
    except Exception as e:
        # Final fallback: try to create a minimal manual QASM-like string
        raise RuntimeError("No available QASM exporter (tried qasm3.dumps and circuit.qasm()). "
                           "Check your Qiskit installation.") from e

# ---------------------------
# Utility: safe draw (mpl fallback to text)
# ---------------------------
def safe_draw(qc):
    try:
        qc.draw('mpl')
        plt.show()
    except Exception as e:
        print("⚠️ Matplotlib drawer failed or missing dependency; showing text diagram instead:\n")
        print(qc.draw())

# ---------------------------
# 1) 2-qubit Bell circuit
# ---------------------------
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

print("=== Drawing 2-qubit circuit ===")
safe_draw(qc)

# QASM export (robust)
try:
    qasm_code = get_qasm(qc)
    print("\n===== Exported QASM =====")
    print(qasm_code)
except RuntimeError as e:
    print("Failed to export QASM:", e)

# Simulate
sim = AerSimulator()
t_qc = transpile(qc, sim)
job = sim.run(t_qc, shots=1024)
result = job.result()
counts = result.get_counts()
print("\n===== Simulation Results (2-qubit) =====")
print(counts)
plot_histogram(counts)
plt.show()

# ---------------------------
# Save QASM string to file (if exported successfully)
# ---------------------------
try:
    with open("bell.qasm", "w") as f:
        f.write(qasm_code)
    print("\nSaved QASM to bell.qasm")
except Exception as e:
    print("Could not save QASM to file:", e)

# ---------------------------
# 2) 3-qubit example with barrier
# ---------------------------
qc2 = QuantumCircuit(3, 3)
qc2.h(0)
qc2.cx(0, 1)
qc2.cx(1, 2)
qc2.barrier()
qc2.measure([0, 1, 2], [0, 1, 2])

print("\n=== Drawing 3-qubit circuit ===")
safe_draw(qc2)

# Export QASM for qc2
try:
    qasm2 = get_qasm(qc2)
    print("\n===== Exported QASM (3-qubit) =====")
    print(qasm2)
except RuntimeError as e:
    print("Failed to export QASM for qc2:", e)

# Simulate qc2
t_qc2 = transpile(qc2, sim)
job2 = sim.run(t_qc2, shots=1024)
result2 = job2.result()
counts2 = result2.get_counts()
print("\n===== Simulation Results (3-qubit) =====")
print(counts2)
plot_histogram(counts2)
plt.show()

# ---------------------------
# Reloading QASM from file (optional)
# If you saved a file above (bell.qasm), show how to read it back.
# Note: qiskit.qasm3.loads or qiskit.qasm3.load may construct circuits from QASM3.
# We'll attempt to read file contents and parse with qasm3.loads if available.
# ---------------------------
try:
    with open("bell.qasm", "r") as f:
        qasm_text = f.read()
    print("\nContents of bell.qasm (first 400 chars):\n")
    print(qasm_text[:400])
    # Try to parse back to circuit using qasm3 if available
    if qasm3_dumps is not None:
        try:
            # qiskit.qasm3.loads is the counterpart of dumps (parse qasm string -> dag/circuit)
            from qiskit.qasm3 import loads as qasm3_loads
            parsed = qasm3_loads(qasm_text)
            print("\nSuccessfully parsed QASM back to a circuit object using qasm3.loads.")
        except Exception as pe:
            print("qasm3.loads not available or failed:", pe)
    else:
        print("qasm3 not available, skipping parsing back.")
except FileNotFoundError:
    print("\nNo bell.qasm file found to reload.")
except Exception as e:
    print("Error while reading/parsing bell.qasm:", e)

# ---------------------------
# Done
# ---------------------------
print("\nCell finished. If you still see NameError for 'dumps', make sure you ran this cell in one run "
      "and that your environment installed the correct qiskit version with qasm3 support.")

=== Drawing 2-qubit circuit ===
⚠️ Matplotlib drawer failed or missing dependency; showing text diagram instead:

     ┌───┐     ┌─┐   
q_0: ┤ H ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
          └───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1 

===== Exported QASM =====
OPENQASM 3.0;
include "stdgates.inc";
bit[2] c;
qubit[2] q;
h q[0];
cx q[0], q[1];
c[0] = measure q[0];
c[1] = measure q[1];


===== Simulation Results (2-qubit) =====
{'11': 508, '00': 516}

Saved QASM to bell.qasm

=== Drawing 3-qubit circuit ===
⚠️ Matplotlib drawer failed or missing dependency; showing text diagram instead:

     ┌───┐           ░ ┌─┐      
q_0: ┤ H ├──■────────░─┤M├──────
     └───┘┌─┴─┐      ░ └╥┘┌─┐   
q_1: ─────┤ X ├──■───░──╫─┤M├───
          └───┘┌─┴─┐ ░  ║ └╥┘┌─┐
q_2: ──────────┤ X ├─░──╫──╫─┤M├
               └───┘ ░  ║  ║ └╥┘
c: 3/═══════════════════╩══╩══╩═
                        0  1  2 

===== Exported QASM (3-qubit) =====
OPENQASM 3.0;
include "stdgates.inc";
bi

In [18]:
# ------------------------------------------------------
# 5. QASM Syntax Notes
# ------------------------------------------------------
print("\n===== QASM Syntax Notes =====")
print("""
In OpenQASM 3.0:
- 'h q[0];'       → Hadamard gate on qubit 0
- 'cx q[0], q[1];'→ CNOT gate (control q0, target q1)
- 'barrier q[0], q[1];' → Prevents reordering of gates
- 'measure q[0] -> c[0];' → Measure qubit 0 to classical bit 0
""")


===== QASM Syntax Notes =====

In OpenQASM 3.0:
- 'h q[0];'       → Hadamard gate on qubit 0
- 'cx q[0], q[1];'→ CNOT gate (control q0, target q1)
- 'barrier q[0], q[1];' → Prevents reordering of gates
- 'measure q[0] -> c[0];' → Measure qubit 0 to classical bit 0

